***
**Downloading CSV and intializing imports**
===

In [ ]:
#@title Download CSV
#@markdown Supported URL types:
#@markdown * Google Drive zip
#@markdown * GitHub raw file

csv_file_link = "https://github.com/codebasics/py/blob/master/ML/9_decision_tree/Exercise/titanic.csv" #@param {type:"string"}
is_csv_from_github_comment = False #@param {type:"boolean"}

import gdown
import requests

csv_name = ''

if 'drive.google.com' in csv_file_link:
  url = csv_file_link
  url = url.replace('/file/d/', '/uc?id=').replace('/view?usp=sharing', '')
  gdown.download(url, quiet=False)

elif 'github.com' in csv_file_link and is_csv_from_github_comment is False:
  url = csv_file_link.replace('github.com', 'raw.githubusercontent.com').replace('/blob', '')
  response = requests.get(url)
  content = response.content
  output = url.split('/')[-1]
  with open(output, 'wb') as f:
      f.write(content)
      print('[', output, ']', 'downloaded succesfully')
      csv_name = output

elif 'github.com' in csv_file_link and is_csv_from_github_comment is True:
  url = csv_file_link
  response = requests.get(url)
  content = response.content
  output = url.split('/')[-1]
  with open(output, 'wb') as f:
      f.write(content)
      print('[', output, ']', 'downloaded successfully')
      csv_name = output

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv(csv_name)

***
**Data Visualization**
===

In [ ]:
# * alt df
df = pd.read_csv('titanic3.csv')

In [ ]:
df.head()

In [ ]:
df.describe(include = 'all')

In [ ]:
sns.heatmap(df.isnull(), cbar = False).set_title("Missing values heatmap")

In this part of the code we can see there are many unique variables, the ones we'll likely use are the following:

TARGET VARIABLE:
SURVIVED

FEATURE VARIABLES:
PCLASS, SEX, AGE, SIBSP(?), FARE,

In [ ]:
df.nunique()